In [168]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.feature_extraction import text  
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict


import sqlite3
cnx = sqlite3.connect('db.sqlite3')

def is_spam(EmailText):
   
    df = pd.read_sql_query("SELECT * FROM application_data_set",cnx)
    
    X = df['EmailText']
    Y = df['Label']
    X_train,X_test,Y_train,Y_test= train_test_split(X,Y, test_size=0.2, random_state=42)
        
    
    vectorizer = text.CountVectorizer(stop_words="english")
    counts = vectorizer.fit_transform(X_train.values)
    spam_nb = MultinomialNB()
    spam_nb.fit(counts,Y_train.values)
    
    labels = ['ham','spam']
    y_train_pred = cross_val_predict(spam_nb, counts, Y_train.values, cv=3)    
    cm = confusion_matrix(Y_train, y_train_pred,labels) 
    print(cm) 
    
    test_vect = vectorizer.transform(np.array([EmailText]))
    predict_test_vect = spam_nb.predict(test_vect)
    new_row = {'Label':predict_test_vect[0],'EmailText':EmailText,'id' :df.shape[0]+1}
    df = df.append(new_row,ignore_index=True)
    df.to_sql(name='application_data_set',con=cnx,if_exists = 'replace',index=False)
    
    return predict_test_vect[0] == 'spam'


In [169]:
print(is_spam("This is to inform you on the release of the EL-GORDO SWEEPSTAKE LOTTERY PROGRAM held on the 29th OF APRIL 2005. Your name is attached to ticket number 025-11-464-992-750 with serial number 2113-05 drew the lucky numbers 13-15-16-21-34-36 which consequently won the lottery in the 3rd category."))

/home/saad/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass labels=['ham', 'spam'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


[[3815   55]
 [  35  635]]
True


In [172]:

def tis_spam(EmailText):
   
    df = pd.read_sql_query("SELECT * FROM application_data_set",cnx)
    
    X = df['EmailText']
    Y = df['Label']
        
    
    vectorizer = text.CountVectorizer(stop_words="english")
    counts = vectorizer.fit_transform(X.values)
    spam_nb = MultinomialNB()
    spam_nb.fit(counts,Y.values)
    
    labels = ['ham','spam']
    y_pred = cross_val_predict(spam_nb, counts, Y.values, cv=3)    
    cm = confusion_matrix(Y, y_pred,labels) 
    print(cm) 
    
    test_vect = vectorizer.transform(np.array([EmailText]))
    predict_test_vect = spam_nb.predict(test_vect)
    new_row = {'Label':predict_test_vect[0],'EmailText':EmailText,'id' :df.shape[0]+1}
    df = df.append(new_row,ignore_index=True)
    df.to_sql(name='application_data_set',con=cnx,if_exists = 'replace',index=False)
    
    return predict_test_vect[0] == 'spam'


In [173]:
print(tis_spam("This is to inform you on the release of the EL-GORDO SWEEPSTAKE LOTTERY PROGRAM held on the 29th OF APRIL 2005. Your name is attached to ticket number 025-11-464-992-750 with serial number 2113-05 drew the lucky numbers 13-15-16-21-34-36 which consequently won the lottery in the 3rd category."))

/home/saad/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)
/home/saad/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass labels=['ham', 'spam'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


[[4780   55]
 [  51  784]]
True
